In [ ]:
from pymongo import MongoClient
from bigchaindb_driver import BigchainDB
import time

In [ ]:
MDB = MongoClient('localhost', 27017)
BDB = BigchainDB('http://localhost:9984')

In [ ]:
MDB.list_database_names()

In [ ]:
mbc = MDB.bigchain

In [ ]:
mbc.list_collection_names()

In [ ]:
for a in mbc.assets.find({'data.asset_type':'university'}):
    print(a)

In [ ]:
for m in mbc.metadata.find({'id': 'a50bb3c754dde6013a4369c1ad6bc7bee325a2591d81a135f9ba7cb3ac279c51'}):
    print(m)

In [ ]:
for a in mbc.assets.find({'data.asset_type':'mark', 'data.student_address': '4'}):
    print(a)
    for m in mbc.metadata.find({'id': a['id']}):
        print(m)

In [ ]:
def _get_marks_by_student(address):
    files = BDB.assets.get(search=address)
    (files, course_data) = _retrieve_course_data(files, address)
    degree_data = _retrieve_degree_data(files)
    mark_data = _retrieve_mark_data(files, course_data)
    return {'degree_data': degree_data, 'mark_data': mark_data}

def _retrieve_degree_data(files):
    if files:
        degree_id = files[0].get('data').get('degree_id')
        degree = BDB.transactions.get(asset_id=degree_id)
        degree_data = {
            'name': degree[0].get('asset').get('data').get('name'),
            'level': degree[0].get('asset').get('data').get('level'),
            'courses': degree[-1].get('metadata').get('courses')
        }
        return degree_data

def _retrieve_course_data(files, address):
    (files, course_ids) = _retrieve_course_ids(files, address)
    course_data = _retrieve_course_information(course_ids)
    return (files, course_data)

def _retrieve_course_ids(files, address):
    courses = []
    for f in files[:]:
        if (f.get('data').get('asset_type') == 'mark') and (f.get('data').get('student_address') == address):
            courses.append(f.get('data').get('course_id'))
        else:
            files.remove(f)
    return (files, list(set(courses)))

def _retrieve_course_information(course_ids):
    course_data = dict()
    for course_id in course_ids:
        course = BDB.transactions.get(asset_id=course_id)
        course_data[course_id] = {
            'name': course[0].get('asset').get('data').get('name'), 
            'lecturer': course[0].get('asset').get('data').get('lecturer'),
            'components': course[-1].get('metadata').get('components')
        }
    return course_data

def _retrieve_mark_data(mark_files, course_data):
    mark_data = dict()
    for f in mark_files:
        course_id = f.get('data').get('course_id')
        mark_type = f.get('data').get('type')
        mark_file = BDB.transactions.get(asset_id=f.get('id'))[-1]
        mark = mark_file.get('metadata').get('mark')
        timestamp = mark_file.get('metadata').get('timestamp')
        weighting = (item for item in course_data.get(course_id).get('components') if item["type"] == mark_type).__next__().get('weighting')
        if mark_data.get(course_id):
            mark_data.get(course_id).get('components')[mark_type] = {
                'mark': mark, 
                'weighting': weighting, 
                'timestamp': timestamp
            }
            if mark_data.get(course_id).get('year') < timestamp[:4]:
                mark_data.get(course_id)['year'] = timestamp[:4]
        else:
            mark_data[course_id] = {
                'name': course_data.get(course_id).get('name'), 
                'lecturer': course_data.get(course_id).get('lecturer'), 
                'year': timestamp[:4],
                'components': {
                    mark_type: {
                        'mark': mark, 
                        'weighting': weighting, 
                        'timestamp': timestamp
                    }
                }
            }
    return mark_data

In [ ]:
t0 = time.time()
_get_marks_by_student('4')
print(time.time() - t0)

In [ ]:
def get_asset_data(_id):
    asset = mbc.assets.find_one({'id': _id})
    return asset['data']

def get_asset_metadata(asset_id):
    transactions = list(mbc.transactions.find({'asset.id': asset_id}))
    if transactions:
        return mbc.metadata.find_one({'id': transactions[-1]['id']})['metadata']
    else:
        return mbc.metadata.find_one({'id': asset_id})['metadata']

In [ ]:
def get_student_mark_assets(student_address):
    return list(mbc.assets.find({'data.asset_type':'mark', 'data.student_address': student_address}))

def process(mark_assets):
    course_ids = list()
    degree_ids = list()
    marks = dict()

    for m in mark_assets:
        course_id = m['data']['course_id']
        degree_id = m['data']['degree_id']
        mark_metadata = get_asset_metadata(m['id'])
        
        if not marks.get(course_id):
            marks[course_id] = dict()
            marks[course_id]['components'] = {m['data']['type']: {'mark': mark_metadata['mark'], 'timestamp': mark_metadata['timestamp']}, 'degree_id': m['data']['degree_id']}
        else:
            marks[course_id]['components'][m['data']['type']] = {'mark': mark_metadata['mark'], 'timestamp': mark_metadata['timestamp'], 'degree_id': m['data']['degree_id']}
            
        course_ids.append(course_id)
        degree_ids.append(degree_id)
        
    course_ids = list(set(course_ids))
    degree_ids = list(set(degree_ids))
    
    return (degree_ids, course_ids, marks)

def add_course_info(marks, course_ids):
    for course_id in course_ids:
        course_asset = get_asset_data(course_id)
        course_metadata = get_asset_metadata(course_id)
        
        if marks.get(course_id).get('year', '1900') < course_metadata['timestamp'][:4]:
            marks.get(course_id)['year'] = course_metadata['timestamp'][:4]
        
        marks[course_id] = {**marks[course_id], **course_asset}
        for c in course_metadata['components']:
            marks[course_id]['components'][c['type']]['weighting'] = c['weighting']
            
    return marks

def add_degree_info(marks, degree_ids):
    degree_data = dict()
    for degree_id in degree_ids:
        degree_data[degree_id] = {**get_asset_data(degree_id), **get_asset_metadata(degree_id)}
    return {'degree_data': degree_data, 'mark_data': marks}

def _get_marks_by_student2(student_address):
    mark_assets = get_student_mark_assets(student_address)
    degree_ids, course_ids, mark_data = process(mark_assets)
    mark_data = add_course_info(mark_data, course_ids)
    mark_data = add_degree_info(mark_data, degree_ids)
    return mark_data

In [ ]:
t0 = time.time()
_get_marks_by_student2('4')
print(time.time() - t0)